<h1> Text Classification</h1>

In [109]:
from pprint import pprint
from time import time
import logging
import numpy as np
from tqdm import tqdm
import spacy
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [61]:
categories = [
    'alt.atheism',
    'talk.religion.misc',
]

In [64]:
data_train = fetch_20newsgroups(subset='train')
data_test = fetch_20newsgroups(subset='test')
print(f"{len(data.filenames)} documents")
print(f"{len(data.target_names)} categories")
print(data.target_names)

11314 documents
20 categories
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [79]:
pipeline1 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('lg', LogisticRegression()),
])

pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('rfc', RandomForestClassifier()),
])


pipeline3 = Pipeline([
    ('cv', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('mnb_clf', MultinomialNB()),
])

pipeline4 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('mnb_clf', MultinomialNB()),
])

pipeline5 = Pipeline([
    ('cv', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('svm_clf', SVC()),
])

pipeline6 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm_clf', SVC()),
])

pipeline7 = Pipeline([
    ('cv', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('dtc_clf', DecisionTreeClassifier()),
])

pipeline8 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('dtc_clf', DecisionTreeClassifier()),
])

In [84]:
parameters1 = {
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'lg__C': [0.1, 1, 100, 1000],
    'lg__penalty': ['l1', 'l2', 'elasticnet'],
    'lg__solver': ['newton-cg', 'lbfgs', 'liblinear']
}

parameters2 = {
     'vect__max_df': [0.5, 0.75, 1.0],
     'vect__max_features': [None, 5000, 10000, 50000],
     'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
     'rfc__n_estimators' : [25,50,75,100,150,200],
 }

parameters3 = {
    'cv__max_df': [0.5, 0.75, 1.0],
    'cv__max_features': [None, 5000, 10000, 50000],
    'cv__ngram_range': ((1, 1), (1, 2)),
    'tfidf__norm': ['l1', 'l2'],
    'tfidf__smooth_idf': [True, False],
    'tfidf__sublinear_tf': [True, False],
    'mnb_clf__alpha': [0.01, 0.1, 0.5, 1.0],
    'mnb_clf__fit_prior': [True, False]
}

parameters4 = {
    'tfidf__stop_words': [None, {'english'}],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1,3)],
    'tfidf__max_df': [0.25, 0.5, 0.75, 1.0],
    'tfidf__max_features': [None, 500, 1000, 2000],
    'tfidf__binary': [True, False],
    'mnb_clf__alpha': [0.01, 0.1, 0.5, 1.0],
    'mnb_clf__fit_prior': [True, False]
}

parameters5 = {
    'cv__max_df': [0.5, 0.75, 1.0],
    'cv__ngram_range': ((1, 1), (1, 2)),
    'tfidf__norm': ['l1', 'l2'],
    'svm_clf__C': [0.5, 1.0],
    'svm_clf__kernel': ['linear', 'poly','rbf', 'sigmoid'],
    'svm_clf__gamma': ['scale', 'auto'],
}

parameters6 = {
    'tfidf__stop_words': ['english'],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__max_df': [0.5, 1.0],
    'svm_clf__C': [0.5, 1.0],
    'svm_clf__kernel': ['linear', 'poly','rbf', 'sigmoid'],
    'svm_clf__gamma': ['scale', 'auto'],
}

parameters7 = {
    'cv__max_df': [0.5, 0.75, 1.0],
    'cv__ngram_range': ((1, 1), (1, 2)),
    'tfidf__norm': ['l1', 'l2'],
    'dtc_clf__criterion': ['gini', 'entropy', 'log_loss'],
    'dtc_clf__max_depth': [10, 20, 30]
}

parameters8 = {
    'tfidf__stop_words': ['english'],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__max_df': [0.5, 1.0],
    'dtc_clf__criterion': ['gini', 'entropy', 'log_loss'],
    'dtc_clf__max_depth': [10, 20, 30]
}

In [43]:
grid_search1 = GridSearchCV(pipeline1, parameters1,n_jobs=-1, verbose=1)

In [44]:
type(data_train.target)

numpy.ndarray

<h3> Tfidf, LogisticRegression</h3>

In [45]:
grid_search1.fit(data_train.data, data_train.target)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/line

/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('lg', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'lg__C': [0.1, 1, 100, 1000],
                         'lg__penalty': ['l1', 'l2', 'elasticnet'],
                         'lg__solver': ['newton-cg', 'lbfgs', 'liblinear'],
                         'tfidf__norm': ('l1', 'l2'),
                         'tfidf__use_idf': (True, False)},
             verbose=1)

In [46]:
print("Best score: %0.3f" % grid_search1.best_score_)

Best score: 0.924


In [48]:
print("Best parameters set:")
best_parameters = grid_search1.best_estimator_.get_params()
for param_name in sorted(parameters1.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best parameters set:
	lg__C: 1000
	lg__penalty: 'l2'
	lg__solver: 'liblinear'
	tfidf__norm: 'l2'
	tfidf__use_idf: True


<h3>CountVectorizer, TfidfTransformer, RandomForestClassifier</h3>

In [56]:
grid_search2 = GridSearchCV(pipeline2, parameters2,n_jobs=-1, verbose=1)

In [57]:
grid_search2.fit(data_train.data, data_train.target)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('rfc', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'rfc__n_estimators': [25, 50, 75, 100, 150, 200],
                         'vect__max_df': [0.5, 0.75, 1.0],
                         'vect__max_features': [None, 5000, 10000, 50000],
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=1)

In [58]:
print("Best score: %0.3f" % grid_search2.best_score_)

Best score: 0.866


In [59]:
print("Best parameters set:")
best_parameters = grid_search2.best_estimator_.get_params()
for param_name in sorted(parameters2.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best parameters set:
	rfc__n_estimators: 200
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (1, 2)


In [82]:
train_data = fetch_20newsgroups(subset='train', categories=categories)

<h3> CountVectorizer, TfidfTransformer, MultinomialNB </h3>

In [83]:
grid_search3 = GridSearchCV(pipeline3, parameters3,n_jobs=-1, verbose=1)
grid_search3.fit(data_train.data, data_train.target)
print("Best score: %0.3f" % grid_search3.best_score_)
print("Best parameters set:")
best_parameters = grid_search3.best_estimator_.get_params()
for param_name in sorted(parameters3.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 1536 candidates, totalling 7680 fits
Best score: 0.951
Best parameters set:
	cv__max_df: 0.5
	cv__max_features: 50000
	cv__ngram_range: (1, 2)
	mnb_clf__alpha: 0.01
	mnb_clf__fit_prior: True
	tfidf__norm: 'l2'
	tfidf__smooth_idf: True
	tfidf__sublinear_tf: True


<h3>TfidfVectorizer, MultinomialNB </h3>

In [86]:
grid_search4 = GridSearchCV(pipeline4, parameters4,n_jobs=-1, verbose=1)
grid_search4.fit(data_train.data, data_train.target)
print("Best score: %0.3f" % grid_search4.best_score_)
print("Best parameters set:")
best_parameters = grid_search4.best_estimator_.get_params()
for param_name in sorted(parameters4.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 1536 candidates, totalling 7680 fits
Best score: 0.954
Best parameters set:
	mnb_clf__alpha: 0.01
	mnb_clf__fit_prior: True
	tfidf__binary: True
	tfidf__max_df: 0.25
	tfidf__max_features: None
	tfidf__ngram_range: (1, 3)
	tfidf__stop_words: None


<h3> CountVectorizer, TfidfTransformer, SVM.SVC </h3>

In [87]:
grid_search5 = GridSearchCV(pipeline5, parameters5,n_jobs=-1, verbose=1)
grid_search5.fit(data_train.data, data_train.target)
print("Best score: %0.3f" % grid_search5.best_score_)
print("Best parameters set:")
best_parameters = grid_search5.best_estimator_.get_params()
for param_name in sorted(parameters5.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 192 candidates, totalling 960 fits
Best score: 0.958
Best parameters set:
	cv__max_df: 0.75
	cv__ngram_range: (1, 2)
	svm_clf__C: 1.0
	svm_clf__gamma: 'scale'
	svm_clf__kernel: 'sigmoid'
	tfidf__norm: 'l2'


<h3> TfidfVectorizer, SVM.SVC </h3>

In [89]:
grid_search6 = GridSearchCV(pipeline6, parameters6,n_jobs=-1, verbose=1)
grid_search6.fit(data_train.data, data_train.target)
print("Best score: %0.3f" % grid_search6.best_score_)
print("Best parameters set:")
best_parameters = grid_search6.best_estimator_.get_params()
for param_name in sorted(parameters6.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 64 candidates, totalling 320 fits
Best score: 0.958
Best parameters set:
	svm_clf__C: 1.0
	svm_clf__gamma: 'scale'
	svm_clf__kernel: 'sigmoid'
	tfidf__max_df: 1.0
	tfidf__ngram_range: (1, 2)
	tfidf__stop_words: 'english'


<h3>CountVectorizer, TfidfTransformer, DecisionTreeClassifier </h3>

In [91]:
grid_search7 = GridSearchCV(pipeline7, parameters7,n_jobs=-1, verbose=1)
grid_search7.fit(data_train.data, data_train.target)
print("Best score: %0.3f" % grid_search7.best_score_)
print("Best parameters set:")
best_parameters = grid_search7.best_estimator_.get_params()
for param_name in sorted(parameters7.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 108 candidates, totalling 540 fits


/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
180 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/tree/_classes.py", li

Best score: 0.878
Best parameters set:
	cv__max_df: 0.75
	cv__ngram_range: (1, 1)
	dtc_clf__criterion: 'gini'
	dtc_clf__max_depth: 30
	tfidf__norm: 'l2'


<h3>TfidfVectorizer, DecisionTreeClassifier</h3>

In [92]:
grid_search8 = GridSearchCV(pipeline8, parameters8,n_jobs=-1, verbose=1)
grid_search8.fit(data_train.data, data_train.target)
print("Best score: %0.3f" % grid_search8.best_score_)
print("Best parameters set:")
best_parameters = grid_search8.best_estimator_.get_params()
for param_name in sorted(parameters8.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/tree/_classes.py", line

Best score: 0.870
Best parameters set:
	dtc_clf__criterion: 'entropy'
	dtc_clf__max_depth: 30
	tfidf__max_df: 0.5
	tfidf__ngram_range: (1, 1)
	tfidf__stop_words: 'english'


<h3>Tfidf Word2Vec</h3>

In [93]:
tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(data_train.data)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

In [97]:
nlp = spacy.load('en_core_web_lg')

In [98]:
vecs1 = []
data = []
for sent in tqdm(data_train.data):
    doc1 = nlp(sent) # returns vector for each sentence
 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])  #Create empty numpy array
    for word1 in doc1:
    # word2vec
        vec1 = word1.vector       #get a vector for that word using GLOVE
    # fetch idf score
        try:
            idf = word2tfidf[str(word1)]  #getting tfidf value for this word
        except:
            idf = 0       #if not available assign 0, like for stopwords
         # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)

        


100%|█████████████████████████████████████████| 857/857 [02:21<00:00,  6.07it/s]


In [118]:
df = pd.DataFrame(vecs1)

In [124]:
params = {'penalty': ['l1', 'l2'], 'C': [0.1, 1, 100, 1000], 'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],}

grid_search_lr_w2v = GridSearchCV(LogisticRegression(), param_grid= params, cv=5, n_jobs=-1, verbose=1)
grid_search_lr_w2v.fit(df, train_data.target)

print("Best score: %0.3f" % grid_search_lr_w2v.best_score_)

print("Best parameters set:")
best_parameters = grid_search_lr_w2v.best_estimator_.get_params()
for param_name in sorted(params.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpr

/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpr

/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/ekanshpr

Best score: 0.811
Best parameters set:
	C: 1000
	penalty: 'l2'
	solver: 'lbfgs'


/Users/ekanshpruthi/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


<h1> Result Table</h1>

<table>
  <tr>
    <th>Algorithm</th>
    <th>Best Score</th>
    <th>Best Parametes</th>
  </tr>
  <tr>
    <td><h3> Tfidf, LogisticRegression</h3></td>
    <td>0.924</td>
    <td><span>lg__C: 1000 <br>
	lg__penalty: 'l2'<br>
	lg__solver: 'liblinear'<br>
	tfidf__norm: 'l2'<br>
	tfidf__use_idf: True</span></td>
  </tr>
  <tr>
    <td><h3>CountVectorizer, TfidfTransformer, RandomForestClassifier</h3></td>
    <td>0.866</td>
    <td>rfc__n_estimators: 200<br>
	vect__max_df: 0.5<br>
	vect__max_features: None<br>
	vect__ngram_range: (1, 2)</td>
  </tr>
  <tr>
      <td><h3>CountVectorizer, TfidfTransformer, MultinomialNB</h3></td>
    <td>0.951</td>
    <td>cv__max_df: 0.5<br>
	cv__max_features: 50000<br>
	cv__ngram_range: (1, 2)<br>
	mnb_clf__alpha: 0.01<br>
	mnb_clf__fit_prior: True<br>
	tfidf__norm: 'l2'<br>
	tfidf__smooth_idf: True<br>
	tfidf__sublinear_tf: True</td>
  </tr>
  <tr>
    <td><h3>TfidfVectorizer, MultinomialNB </h3></td>
    <td>0.954</td>
    <td>mnb_clf__alpha: 0.01<br>
	mnb_clf__fit_prior: True<br>
	tfidf__binary: True<br>
	tfidf__max_df: 0.25<br>
	tfidf__max_features: None<br>
	tfidf__ngram_range: (1, 3)<br>
	tfidf__stop_words: None</td>
  </tr>
  <tr>
    <td><h3> CountVectorizer, TfidfTransformer, SVM.SVC </h3></td>
    <td>0.958</td>
    <td>cv__max_df: 0.75<br>
	cv__ngram_range: (1, 2)<br>
	svm_clf__C: 1.0<br>
	svm_clf__gamma: 'scale'<br>
	svm_clf__kernel: 'sigmoid'<br>
	tfidf__norm: 'l2'</td>
  </tr>
  <tr>
    <td> <h3> TfidfVectorizer, SVM.SVC </h3></td>
    <td>0.958</td>
    <td>svm_clf__C: 1.0<br>
	svm_clf__gamma: 'scale'<br>
	svm_clf__kernel: 'sigmoid'<br>
	tfidf__max_df: 1.0<br>
	tfidf__ngram_range: (1, 2)<br>
	tfidf__stop_words: 'english'</td>
  </tr>
    <tr>
    <td><h3>CountVectorizer, TfidfTransformer, DecisionTreeClassifier </h3> </td>
    <td>0.870</td>
    <td>dtc_clf__criterion: 'entropy'<br>
	dtc_clf__max_depth: 30<br>
	tfidf__max_df: 0.5<br>
	tfidf__ngram_range: (1, 1)<br>
	tfidf__stop_words: 'english'</td>
  </tr>
    <tr>
    <td><h3>TfidfVectorizer, DecisionTreeClassifier</h3> </td>
    <td>0.870</td>
    <td>dtc_clf__criterion: 'entropy'<br>
	dtc_clf__max_depth: 30<br>
	tfidf__max_df: 0.5<br>
	tfidf__ngram_range: (1, 1)<br>
	tfidf__stop_words: 'english'</td>
  </tr>
    <tr>
    <td><h3>Tfidf Word2Vec Logistic Regression</h3> </td>
    <td>0.811</td>
    <td>C: 1000<br>
	penalty: 'l2'<br>
	solver: 'lbfgs'</td>
  </tr>
</table>